In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

from SVGFuncAnimation import *

plt.rcParams['animation.frame_format'] = 'svg'
plt.rcParams['animation.html'] = 'jshtml'
plt.rcParams['animation.embed_limit'] = 2 ** 128

# Simple Line Plot Animation

In [2]:
def get_anim(constructor, size=100):
    np.random.seed(0)

    def update_line(num, data, line):
        line.set_data(range(num), data[:num])
        return line,

    fig = plt.figure()
    data = np.random.rand(size)
    l, = plt.plot([], [], 'r-')
    plt.xlim(0, size - 1)
    plt.ylim(0, 1)

    anim = constructor(fig, update_line, range(1, size + 1), fargs=(data, l))
    plt.close(fig)
    
    return anim

In [3]:
# Test the standard FuncAnimation + HTMLWriter (implicitly used by to_jshtml)
%time print(f'Size: {len(get_anim(FuncAnimation, size=1000).to_jshtml()):,}')

Size: 31,624,049
Wall time: 29 s


In [4]:
# Test SVGFuncAnimation (which uses it's own, built-in Writer)
%time print(f'Size: {len(get_anim(SVGFuncAnimation, size=1000).to_jshtml()):,}')

Size: 11,664,481
Wall time: 932 ms


In [5]:
get_anim(SVGFuncAnimation)

# Simple Scatter Plot Animation

In [6]:
def get_anim(constructor):
    np.random.seed(123)

    def update_plot(i, data, scat):
        scat.set_array(data[i])
        offset = np.random.random((numpoints, 2)) - 0.5
        scat.set_offsets(np.stack([x, y]).T + 0.2*offset)
        return scat,

    numframes = 100
    numpoints = 10
    color_data = np.random.random((numframes, numpoints))
    x, y, c = np.random.random((3, numpoints))

    fig = plt.figure()
    scat = plt.scatter(x, y, c=c, s=100)
    plt.xlim(-0.5, 1.5)
    plt.ylim(-0.5, 1.5)

    anim = constructor(fig, update_plot, frames=range(numframes), fargs=(color_data, scat))
    plt.close(fig)
    return anim

In [7]:
# Test the standard FuncAnimation + HTMLWriter (implicitly used by to_jshtml)
%time print(f'Size: {len(get_anim(FuncAnimation).to_jshtml()):,}')

Size: 2,312,424
Wall time: 3.98 s


In [8]:
# Test SVGFuncAnimation (which uses it's own, built-in Writer)
%time print(f'Size: {len(get_anim(SVGFuncAnimation).to_jshtml()):,}')

Size: 227,236
Wall time: 202 ms


In [9]:
get_anim(SVGFuncAnimation)

# Pendulum example
taken from: https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/

In [10]:
from numpy import sin, cos
import scipy.integrate as integrate

class DoublePendulum:
    """Double Pendulum Class

    init_state is [theta1, omega1, theta2, omega2] in degrees,
    where theta1, omega1 is the angular position and velocity of the first
    pendulum arm, and theta2, omega2 is that of the second pendulum arm
    """
    def __init__(self,
                 init_state = [120, 0, -20, 0],
                 L1=1.0,  # length of pendulum 1 in m
                 L2=1.0,  # length of pendulum 2 in m
                 M1=1.0,  # mass of pendulum 1 in kg
                 M2=1.0,  # mass of pendulum 2 in kg
                 G=9.8,  # acceleration due to gravity, in m/s^2
                 origin=(0, 0)): 
        self.init_state = np.asarray(init_state, dtype='float')
        self.params = (L1, L2, M1, M2, G)
        self.origin = origin
        self.time_elapsed = 0

        self.state = self.init_state * np.pi / 180.
    
    def position(self):
        """compute the current x,y positions of the pendulum arms"""
        (L1, L2, M1, M2, G) = self.params

        x = np.cumsum([self.origin[0],
                       L1 * sin(self.state[0]),
                       L2 * sin(self.state[2])])
        y = np.cumsum([self.origin[1],
                       -L1 * cos(self.state[0]),
                       -L2 * cos(self.state[2])])
        return (x, y)

    def energy(self):
        """compute the energy of the current state"""
        (L1, L2, M1, M2, G) = self.params

        x = np.cumsum([L1 * sin(self.state[0]),
                       L2 * sin(self.state[2])])
        y = np.cumsum([-L1 * cos(self.state[0]),
                       -L2 * cos(self.state[2])])
        vx = np.cumsum([L1 * self.state[1] * cos(self.state[0]),
                        L2 * self.state[3] * cos(self.state[2])])
        vy = np.cumsum([L1 * self.state[1] * sin(self.state[0]),
                        L2 * self.state[3] * sin(self.state[2])])

        U = G * (M1 * y[0] + M2 * y[1])
        K = 0.5 * (M1 * np.dot(vx, vx) + M2 * np.dot(vy, vy))

        return U + K

    def dstate_dt(self, state, t):
        """compute the derivative of the given state"""
        (M1, M2, L1, L2, G) = self.params

        dydx = np.zeros_like(state)
        dydx[0] = state[1]
        dydx[2] = state[3]

        cos_delta = cos(state[2] - state[0])
        sin_delta = sin(state[2] - state[0])

        den1 = (M1 + M2) * L1 - M2 * L1 * cos_delta * cos_delta
        dydx[1] = (M2 * L1 * state[1] * state[1] * sin_delta * cos_delta
                   + M2 * G * sin(state[2]) * cos_delta
                   + M2 * L2 * state[3] * state[3] * sin_delta
                   - (M1 + M2) * G * sin(state[0])) / den1

        den2 = (L2 / L1) * den1
        dydx[3] = (-M2 * L2 * state[3] * state[3] * sin_delta * cos_delta
                   + (M1 + M2) * G * sin(state[0]) * cos_delta
                   - (M1 + M2) * L1 * state[1] * state[1] * sin_delta
                   - (M1 + M2) * G * sin(state[2])) / den2
        
        return dydx

    def step(self, dt):
        """execute one time step of length dt and update state"""
        self.state = integrate.odeint(self.dstate_dt, self.state, [0, dt])[1]
        self.time_elapsed += dt

def get_anim(constructor):
    # set up initial state and global variables
    pendulum = DoublePendulum([180., 0.0, -20., 0.0])
    dt = 1./30 # 30 fps

    # set up figure and animation
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=(-2, 2), ylim=(-2, 2))
    ax.grid()

    line, = ax.plot([], [], 'o-', lw=2)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    
    def init():
        """initialize animation"""
        line.set_data([], [])
        time_text.set_text(' ')
        energy_text.set_text(' ')
        return line, time_text, energy_text

    def animate(i):
        """perform animation step"""
        nonlocal pendulum, dt
        pendulum.step(dt)

        line.set_data(*pendulum.position())
        time_text.set_text('time = %.1f' % pendulum.time_elapsed)
        energy_text.set_text('energy = %.3f J' % pendulum.energy())
        return line, time_text, energy_text
    
    anim = constructor(fig, animate, frames=range(300), interval=35, init_func=init)
    plt.close(fig)
    return anim

In [11]:
# Test the standard FuncAnimation + HTMLWriter (implicitly used by to_jshtml)
%time print(f'Size: {len(get_anim(FuncAnimation).to_jshtml()):,}')

Size: 9,228,673
Wall time: 12.1 s


In [12]:
# Test SVGFuncAnimation (which uses it's own, built-in Writer)
%time print(f'Size: {len(get_anim(SVGFuncAnimation).to_jshtml()):,}')

Size: 737,614
Wall time: 906 ms


In [13]:
get_anim(SVGFuncAnimation)